In [1]:

from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define source and destination folders
source_folder = '/content/drive/MyDrive/Etsy/'  # Change this if your CSVs are in a subfolder
destination_folder = '/content/'

# Create destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Copy all CSV files
for filename in os.listdir(source_folder):
    if filename.endswith('.csv'):
        full_file_name = os.path.join(source_folder, filename)
        shutil.copy(full_file_name, destination_folder)

print(f"All CSV files copied to: {destination_folder}")


!pip install reportlab
from pathlib import Path
import os
import pandas as pd
idx=pd.IndexSlice

Mounted at /content/drive
All CSV files copied to: /content/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00


In [2]:
year='2025'
bank_name='Inspired (0200)'
from datetime import datetime

# Get current date
current_date1 = datetime.now()

# Format as mm/dd/yy
current_date = current_date1.strftime("%m/%d/%y")
print(current_date)

11/14/25


In [3]:
orders_filename='EtsySoldOrders%s.csv'%year
orders_items_filename='EtsySoldOrderItems%s.csv'%year

In [4]:
df_orders_raw=pd.read_csv(orders_filename)

In [5]:
df_orders=df_orders_raw.set_index(['Sale Date','Order ID']).loc[:,['Full Name','Order Value','Shipping','Discount Amount','Order Total']]


In [6]:
df_orders_items_raw=pd.read_csv(orders_items_filename)

In [7]:
df_orders_items=df_orders_items_raw.set_index(['Sale Date','Order ID']).loc[:,['Item Name','Quantity','Price']]

In [8]:
item_list=[]
for order in df_orders.index.get_level_values(1):

    # Group by Item Name and Price to keep separate prices
    df_grouped = (
        df_orders_items.loc[idx[:, order], :]
        .groupby(['Item Name', 'Price'], as_index=False)
        .agg({'Quantity': 'sum'})
    )

    # If same Item Name appears with different prices, append numbering
    df_grouped['Display Name'] = df_grouped.groupby('Item Name').cumcount().add(1).astype(str)
    df_grouped['Display Name'] = df_grouped.apply(
        lambda x: f"{x['Item Name']} ({x['Display Name']})" if df_grouped['Item Name'].duplicated(keep=False)[x.name] else x['Item Name'],
        axis=1
    )
    df_grouped['Item Name']=df_grouped['Display Name']
    df_grouped=df_grouped.loc[:,df_grouped.columns[:-1]]
    items=df_grouped.set_index('Item Name').to_dict()
    item_list.append(items)
df_orders.loc[:,'Item Details']=item_list

In [9]:
dfs=[]
months=[2,3,4,5,6,7,8,9,10]
months=[8]
for month in months:
    tmp_filename='etsy_statement_%s_%d.csv'%(year,month)
    if(os.path.exists(tmp_filename)):
        df_temp=pd.read_csv(tmp_filename)
        dfs.append(df_temp)
df_statements=pd.concat(dfs)

In [10]:
df_orders_fees=df_statements[df_statements.Info.str.startswith('Order',na=False)].set_index('Info').loc[:,['Title','Fees & Taxes']]
df_orders_fees.index=df_orders_fees.index.str.replace('Order #', '', regex=False).astype(int)

In [11]:
item_list=[]

# Make index names unique by appending (1), (2), etc.
def make_unique_index(df):
    counts = {}
    new_index = []
    for idx in df.index:
        if idx in counts:
            counts[idx] += 1
            new_index.append(f"{idx}({counts[idx]})")
        else:
            counts[idx] = 0
            new_index.append(idx)
    df.index = new_index
    return df

for order in df_orders.index.get_level_values(1):
    if(order in df_orders_fees.index):
        items=make_unique_index(df_orders_fees.loc[order,:].set_index('Title'))['Fees & Taxes'].to_dict()
        item_list.append((order,items))
df_orders_fees_aligned=pd.DataFrame(item_list,columns=['Order Id','Fee details']).set_index('Order Id')
df_orders_aligned=df_orders.loc[idx[:,df_orders_fees_aligned.index],:]
df_full_orders_with_fees=df_orders_aligned.reset_index().set_index('Order ID').join(df_orders_fees_aligned).reset_index().set_index(['Sale Date','Order ID'])


In [12]:
df_other_trans=df_statements[~df_statements.Info.str.startswith('Order',na=False)]


In [13]:
df_full_orders_with_fees.loc[:,'Sales Tax']=df_full_orders_with_fees['Order Total'] - (df_full_orders_with_fees['Order Value'] -df_full_orders_with_fees['Discount Amount'])- df_full_orders_with_fees['Shipping']

In [14]:
df_full_orders_with_fees

Full Name  Order Value  Shipping  \
Sale Date Order ID                                                 
08/31/25  3786441109     Brooke Botwinick         12.0      4.58   
          3786045109         Sheila Rubin         16.0      4.89   
          3786039975           Abby Craig         16.0      5.02   
08/30/25  3784673003    Deborah Rosenberg         48.0      4.80   
08/29/25  3783651967         Yonah Haring         28.0      5.41   
08/26/25  3781506829         Golda Kaplan         84.0     14.20   
08/25/25  3780788463      Nanette G Sacks         72.0      6.02   
          3780142919        Meganne Gould         48.0      0.00   
08/24/25  3776558624     Rachel Warshower         56.0     12.75   
08/23/25  3775702196        Lydia Schwetz         48.0      7.20   
08/20/25  3776248707    Linda Rose Bolton         64.0     12.75   
08/19/25  3771775252       Cynthia F Nash         32.0      6.02   
08/17/25  3773375153         Hilary Davis         32.0      5.83   
08/14/25  3766376116         Jill Harries         36.0      7.10   
08/13/25  3765076856       Edwina Michaud         48.0      7.10   
08/11/25  3762913564    Kristen Broughton         12.0      4.89   
08/10/25  3766478769          Jill Teslow         22.0      4.89   
08/09/25  3761335772     Kristina Calamia         32.0      9.22   
08/08/25  3760559888       Jamie Kitchens         16.0      4.80   
          3765112673  Judith E. F. Gamboa         24.0     10.04   
08/05/25  3757330314      Janette Jaffrey         22.0      4.89   
08/04/25  3760667567   stephanie Burnside         22.0      0.00   
08/02/25  3759003319       Barbara Litwin         32.0      6.02   

                      Discount Amount  Order Total  \
Sale Date Order ID                                   
08/31/25  3786441109              0.0        17.75   
          3786045109              0.0        22.35   
          3786039975              0.0        22.65   
08/30/25  3784673003              9.6        46.43   
08/29/25  3783651967              0.0        36.04   
08/26/25  3781506829             16.8        87.10   
08/25/25  3780788463             14.4        68.95   
          3780142919              9.6        48.09   
08/24/25  3776558624             11.2        61.36   
08/23/25  3775702196              9.6        48.38   
08/20/25  3776248707             12.8        67.02   
08/19/25  3771775252              0.0        40.68   
08/17/25  3773375153              0.0        40.57   
08/14/25  3766376116              7.2        38.06   
08/13/25  3765076856              9.6        48.69   
08/11/25  3762913564              1.2        16.55   
08/10/25  3766478769              0.0        28.43   
08/09/25  3761335772              0.0        43.70   
08/08/25  3760559888              0.0        22.16   
          3765112673              0.0        36.34   
08/05/25  3757330314              0.0        29.51   
08/04/25  3760667567              0.0        27.02   
08/02/25  3759003319              0.0        40.02   

                                                           Item Details  \
Sale Date Order ID                                                        
08/31/25  3786441109  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
          3786045109  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
          3786039975  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/30/25  3784673003  {'Price': {'Rosh Hashanah Pomegranate napkin r...   
08/29/25  3783651967  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/26/25  3781506829  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/25/25  3780788463  {'Price': {'Apple Tree of Life Rosh Hashanah n...   
          3780142919  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/24/25  3776558624  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/23/25  3775702196  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/20/25  3776248707  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/19

In [15]:
def conv_to_etsy_invoice_row(row):
    perc=row['Discount Amount']/row['Order Value']
    items=row['Item Details']
    index=0
    dfs=[]
    all_keys=list(items['Price'].keys())
    key1=all_keys[index]
    df_tmp={'InvoiceNo':row.name[1],
          'Customer':row['Full Name'],
          'InvoiceDate':row.name[0],
          'DueDate':row.name[0],
          'Shipping Charge':row['Shipping'],
          'Item Description':key1,
          'Item Rate':items['Price'][key1]*(1-perc),
          'Item Qty':items['Quantity'][key1],
          'ItemAmount':items['Quantity'][key1]*items['Price'][key1]*(1-perc)
          }
    dfs.append(df_tmp)
    for index in range(1,len(items['Quantity'].keys()),1):
        key1=all_keys[index]
        df_tmp={'InvoiceNo':row.name[1],
              'Customer':'',
              'InvoiceDate':'',
              'DueDate':'',
              'Shipping Charge':'',
              'Item Description':key1,
              'Item Rate':items['Price'][key1]*(1-perc),
              'Item Qty':items['Quantity'][key1],
              'ItemAmount':items['Quantity'][key1]*items['Price'][key1]*(1-perc)
            }
        dfs.append(df_tmp)
    df_tmp={'InvoiceNo':row.name[1],
          'Customer':'',
          'InvoiceDate':'',
          'DueDate':'',
          'Shipping Charge':'',
          'Item Description':'Sales Tax',
          'Item Rate':row['Sales Tax'],
          'Item Qty':'1',
          'ItemAmount':row['Sales Tax']
        }
    dfs.append(df_tmp)
    return pd.DataFrame(dfs).set_index('InvoiceNo')


dfs=[]
for n,row in df_full_orders_with_fees.iterrows():
    dfs.append(conv_to_etsy_invoice_row(row))
invoice_dfs=pd.concat(dfs)
invoice_dfs.to_csv('invoices.csv')

In [16]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_RIGHT
import os

styles = getSampleStyleSheet()

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_RIGHT
import os

styles = getSampleStyleSheet()

def generate_order_expense(path, row):
    filename = f"order_expense_{row.name[1]}.pdf"
    doc = SimpleDocTemplate(os.path.join(path, filename), pagesize=letter)
    elements = []

    # Header
    elements.append(Paragraph("Order Expense", styles['Title']))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(
        f"Order ID: {row.name[1]}<br/>Date: {row.name[0]}<br/>Payee: {'etsy'}",
        styles['Normal']
    ))
    elements.append(Spacer(1, 12))

    # Fee Details
    fee_details = row['Fee details']  # This should be a dict like the one you provided
    data = [["Fee Description", "Amount"]]
    total_expense = 0.0

    for desc, amount_str in fee_details.items():
        desc = 'etsy ' + desc
        clean_amount_str = amount_str.replace('$', '').replace(',', '').replace('−', '-').replace('–', '-')
        amount = float(clean_amount_str)
        total_expense += amount
        if amount < 0:
            formatted_amount = f"${-amount:.2f}"
        else:
            formatted_amount = f"-${amount:.2f}"
        data.append([Paragraph(desc, styles['Normal']), formatted_amount])

    # Fee Table
    table = Table(data, colWidths=[400, 80])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('VALIGN', (0,1), (-1,-1), 'TOP'),
        ('ALIGN', (1,1), (-1,-1), 'RIGHT')
    ]))
    elements.append(table)
    elements.append(Spacer(1, 12))

    # Total Expense
    right_align_style = ParagraphStyle(
        name='RightAlign',
        parent=styles['Normal'],
        alignment=TA_RIGHT,
        fontName='Helvetica-Bold'
    )


    if total_expense < 0:
        formatted_total = f"${-total_expense:.2f}"
    else:
        formatted_total = f"-${total_expense:.2f}"

    total_table = Table(
        [[Paragraph("<b>Total Expense</b>", styles['Normal']),
          Paragraph(f"<b>{formatted_total}</b>", right_align_style)]],
        colWidths=[400, 80]
    )
    total_table.setStyle(TableStyle([
        ('ALIGN', (1,0), (1,0), 'RIGHT'),
        ('FONTNAME', (0,0), (-1,-1), 'Helvetica-Bold')
    ]))
    elements.append(total_table)

    # Build PDF
    doc.build(elements)
    print(f"Saved: {filename}")

path='order_expenses'
import os
import shutil

def prepare_path(path):
    if os.path.exists(path):
        # Clear contents of the directory
        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # remove file or link
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # remove directory
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        # Create the directory
        os.makedirs(path)
prepare_path(path)
for n,row in df_full_orders_with_fees.iterrows():
    generate_order_expense(path,row)
shutil.make_archive(os.path.join(path), 'zip', path)

Saved: order_expense_3786441109.pdf
Saved: order_expense_3786045109.pdf
Saved: order_expense_3786039975.pdf
Saved: order_expense_3784673003.pdf
Saved: order_expense_3783651967.pdf
Saved: order_expense_3781506829.pdf
Saved: order_expense_3780788463.pdf
Saved: order_expense_3780142919.pdf
Saved: order_expense_3776558624.pdf
Saved: order_expense_3775702196.pdf
Saved: order_expense_3776248707.pdf
Saved: order_expense_3771775252.pdf
Saved: order_expense_3773375153.pdf
Saved: order_expense_3766376116.pdf
Saved: order_expense_3765076856.pdf
Saved: order_expense_3762913564.pdf
Saved: order_expense_3766478769.pdf
Saved: order_expense_3761335772.pdf
Saved: order_expense_3760559888.pdf
Saved: order_expense_3765112673.pdf
Saved: order_expense_3757330314.pdf
Saved: order_expense_3760667567.pdf
Saved: order_expense_3759003319.pdf


'/content/order_expenses.zip'

In [17]:
df_full_orders_with_fees

Full Name  Order Value  Shipping  \
Sale Date Order ID                                                 
08/31/25  3786441109     Brooke Botwinick         12.0      4.58   
          3786045109         Sheila Rubin         16.0      4.89   
          3786039975           Abby Craig         16.0      5.02   
08/30/25  3784673003    Deborah Rosenberg         48.0      4.80   
08/29/25  3783651967         Yonah Haring         28.0      5.41   
08/26/25  3781506829         Golda Kaplan         84.0     14.20   
08/25/25  3780788463      Nanette G Sacks         72.0      6.02   
          3780142919        Meganne Gould         48.0      0.00   
08/24/25  3776558624     Rachel Warshower         56.0     12.75   
08/23/25  3775702196        Lydia Schwetz         48.0      7.20   
08/20/25  3776248707    Linda Rose Bolton         64.0     12.75   
08/19/25  3771775252       Cynthia F Nash         32.0      6.02   
08/17/25  3773375153         Hilary Davis         32.0      5.83   
08/14/25  3766376116         Jill Harries         36.0      7.10   
08/13/25  3765076856       Edwina Michaud         48.0      7.10   
08/11/25  3762913564    Kristen Broughton         12.0      4.89   
08/10/25  3766478769          Jill Teslow         22.0      4.89   
08/09/25  3761335772     Kristina Calamia         32.0      9.22   
08/08/25  3760559888       Jamie Kitchens         16.0      4.80   
          3765112673  Judith E. F. Gamboa         24.0     10.04   
08/05/25  3757330314      Janette Jaffrey         22.0      4.89   
08/04/25  3760667567   stephanie Burnside         22.0      0.00   
08/02/25  3759003319       Barbara Litwin         32.0      6.02   

                      Discount Amount  Order Total  \
Sale Date Order ID                                   
08/31/25  3786441109              0.0        17.75   
          3786045109              0.0        22.35   
          3786039975              0.0        22.65   
08/30/25  3784673003              9.6        46.43   
08/29/25  3783651967              0.0        36.04   
08/26/25  3781506829             16.8        87.10   
08/25/25  3780788463             14.4        68.95   
          3780142919              9.6        48.09   
08/24/25  3776558624             11.2        61.36   
08/23/25  3775702196              9.6        48.38   
08/20/25  3776248707             12.8        67.02   
08/19/25  3771775252              0.0        40.68   
08/17/25  3773375153              0.0        40.57   
08/14/25  3766376116              7.2        38.06   
08/13/25  3765076856              9.6        48.69   
08/11/25  3762913564              1.2        16.55   
08/10/25  3766478769              0.0        28.43   
08/09/25  3761335772              0.0        43.70   
08/08/25  3760559888              0.0        22.16   
          3765112673              0.0        36.34   
08/05/25  3757330314              0.0        29.51   
08/04/25  3760667567              0.0        27.02   
08/02/25  3759003319              0.0        40.02   

                                                           Item Details  \
Sale Date Order ID                                                        
08/31/25  3786441109  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
          3786045109  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
          3786039975  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/30/25  3784673003  {'Price': {'Rosh Hashanah Pomegranate napkin r...   
08/29/25  3783651967  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/26/25  3781506829  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/25/25  3780788463  {'Price': {'Apple Tree of Life Rosh Hashanah n...   
          3780142919  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/24/25  3776558624  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/23/25  3775702196  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/20/25  3776248707  {'Price': {'Rosh Hashana Pomegranate Tea Light...   
08/19

In [18]:
df_other_trans_filt1=df_other_trans[~df_other_trans.Title.str.startswith('Payment for Order #',na=False)]

In [19]:
df_other_trans_filt2=df_other_trans_filt1[~df_other_trans_filt1.Title.str.contains('sent to your bank account',na=False)]

In [20]:
df_other_trans_filt2.Title.unique()

array(['Listing fee', 'Etsy Ads', 'USPS shipping label',
       'Adjustment for USPS shipping label',
       'Credit for USPS shipping label adjustment',
       'Partial refund for Order #3761335772',
       'Partial refund for Order #3765112673'], dtype=object)

In [21]:
df_shipping=df_other_trans_filt2[df_other_trans_filt2.Title.str.contains('USPS')]

In [22]:
df_shipping

,Date,Type,Title,Info,Currency,Amount,Fees & Taxes,Net,Tax Details
19,"August 30, 2025",Shipping,USPS shipping label,Label #281093639258,USD,--,-$5.70,-$5.70,--
28,"August 29, 2025",Shipping,USPS shipping label,Label #282355887805,USD,--,-$6.53,-$6.53,--
38,"August 28, 2025",Shipping,USPS shipping label,Label #280910341150,USD,--,-$12.11,-$12.11,--
40,"August 28, 2025",Shipping,Adjustment for USPS shipping label,Adjustment #281683095195,USD,--,-$3.83,-$3.83,--
41,"August 27, 2025",Shipping,USPS shipping label,Label #282189306675,USD,--,-$16.92,-$16.92,--
42,"August 27, 2025",Shipping,Adjustment for USPS shipping label,Adjustment #281458228063,USD,--,-$3.48,-$3.48,--
63,"August 25, 2025",Shipping,USPS shipping label,Label #280541031626,USD,--,-$7.10,-$7.10,--
64,"August 25, 2025",Shipping,USPS shipping label,Label #281939436641,USD,--,-$12.75,-$12.75,--
72,"August 24, 2025",Shipping,USPS shipping label,Label #280515812926,USD,--,-$7.20,-$7.20,--
94,"August 21, 2025",Shipping,USPS shipping label,Label #281683095195,USD,--,-$12.75,-$12.75,--


In [23]:
df_listing=df_other_trans_filt2[df_other_trans_filt2.Title.str.contains('Listing fee')]

In [24]:
df_listing

,Date,Type,Title,Info,Currency,Amount,Fees & Taxes,Net,Tax Details
5,"August 31, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
11,"August 31, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
17,"August 31, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
26,"August 30, 2025",Fee,Listing fee,Listing #4338307161,USD,--,-$0.20,-$0.20,--
27,"August 30, 2025",Fee,Listing fee,Listing #4338307161,USD,--,-$0.60,-$0.60,--
36,"August 29, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
37,"August 29, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
50,"August 26, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
51,"August 26, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.20,-$0.20,--
52,"August 26, 2025",Fee,Listing fee,Listing #4331939325,USD,--,-$0.40,-$0.40,--


In [25]:
df_ads=df_other_trans_filt2[df_other_trans_filt2.Title.str.contains('Etsy Ads')]

In [26]:
df_ads

,Date,Type,Title,Info,Currency,Amount,Fees & Taxes,Net,Tax Details
18,"August 31, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 30...,USD,--,-$3.94,-$3.94,--
20,"August 30, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 29...,USD,--,-$3.98,-$3.98,--
29,"August 29, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 28...,USD,--,-$3.95,-$3.95,--
39,"August 28, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 27...,USD,--,-$3.94,-$3.94,--
43,"August 27, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 26...,USD,--,-$3.66,-$3.66,--
54,"August 26, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 25...,USD,--,-$2.93,-$2.93,--
62,"August 25, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 24...,USD,--,-$3.98,-$3.98,--
84,"August 24, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 23...,USD,--,-$3.87,-$3.87,--
92,"August 23, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 22...,USD,--,-$2.80,-$2.80,--
93,"August 22, 2025",Marketing,Etsy Ads,Bill for click-throughs to your shop on Aug 21...,USD,--,-$2.98,-$2.98,--


In [27]:
def create_journal_entry(journal_num,journal_date,bank_account,account_name,row,first=False):
    num=row['Net']
    num1=float(num.replace('$',''))
    if num1>0:
        Debits=str(abs(num1))
        Credits=''
    else:
        Credits=str(abs(num1))
        Debits=''
    desc='%s %s %s'%(row['Date'],row['Title'],row['Info'])
    if(first):
        journal_date2=journal_date
    else:
        journal_date2=''
    df_tmp1={'*JournalNo':journal_num,
            '*JournalDate':journal_date2,
            '*AccountName':bank_account,
            '*Debits':Debits,
            '*Credits':Credits,
            'Description':desc}
    df_tmp2={'*JournalNo':journal_num,
            '*JournalDate':'',
            '*AccountName':account_name,
            '*Debits':Credits,
            '*Credits':Debits,
            'Description':desc}
    return pd.DataFrame([df_tmp1,df_tmp2])
def create_journal(journal_num,journal_date,bank_account,account_name,df):
    dfs=[]
    first=True
    for n,row in df.iterrows():
        dfs.append(create_journal_entry(journal_num,journal_date,bank_account,account_name,row,first=first))
        first=False
    return pd.concat(dfs)

In [28]:
import pandas as pd
import calendar
def split_to_months(df):
  # Ensure 'Date' column is in datetime format
  df=df.copy()
  df['Date'] = pd.to_datetime(df['Date'])

  # Group by year and month
  grouped = df.groupby(df['Date'].dt.to_period('M'))

  # Create list of tuples: (df_month, last_calendar_date in mm/dd/yy)
  monthly_dfs = []
  for period, group in grouped:
      df_month = group.copy()
      year = period.year
      month = period.month
      last_day = calendar.monthrange(year, month)[1]
      last_date = pd.Timestamp(year=year, month=month, day=last_day).strftime('%m/%d/%y')
      monthly_dfs.append((df_month, last_date))
  return monthly_dfs
def create_journals_from_splitted_dfs(spl_dfs,name,bank,account_name):
  jrnls=[]
  for df,date in spl_dfs:
    jrnls.append(create_journal('%s-%s'%(name,date),date,bank,account_name,df))
  jrnl=pd.concat(jrnls)
  return jrnl
def create_jrnl_comb(df,name,bank,account_name):
  spl_dfs=split_to_months(df)
  return create_journals_from_splitted_dfs(spl_dfs,name,bank,account_name)



In [29]:
create_jrnl_comb(df_shipping,'shipping',bank_name,'Etsy:Shipping').to_csv('shipping_jrnl2.csv')
create_jrnl_comb(df_ads,'ads',bank_name,'Etsy:Ads').to_csv('ads_jrnl2.csv')
create_jrnl_comb(df_listing,'listing',bank_name,'Etsy:Listing').to_csv('listing_jrnl2.csv')